**Importar Librerías**

---

In [5]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np # Para np.nan
import os # Importar el módulo os para manejar rutas de archivos


---

**Conexión a Base de Datos**

---

In [7]:
from db_connection import get_connection
from db_connection import get_engine

ModuleNotFoundError: No module named 'db_connection'

---

----

**Limpieza y Análisis de Datosde Cada una de las Bases de Datos**

---

***1. Tabla accounting_account_balances***

---

In [ ]:
# 1. Reconectar (o asegurar que la conexión esté activa)
try:
    conexion = get_connection() # Obtener una nueva conexión o reutilizar si get_connection maneja eso
    cursor = conexion.cursor()

    # 2. Ejecutar la consulta para describir la tabla
    cursor.execute("DESCRIBE accounting_account_balances;")
    columns_info = cursor.fetchall()

    print("Columnas de la tabla 'accounting_account_balances':")
    for column in columns_info:
        print(f"  - {column[0]} ({column[1]})")

except Exception as e:
    print(f"Ocurrió un error: {e}")



✅ Conexión exitosa a la base de datos
Columnas de la tabla 'accounting_account_balances':
  - id (bigint unsigned)
  - code (double(16,15))
  - accounting_id (bigint unsigned)
  - name (varchar(255))
  - initial_balance (double(18,6))
  - final_balance (double(18,6))
  - debit_movement (double(18,6))
  - credit_movement (double(18,6))
  - third_party_type_id (varchar(50))
  - third_party_id (bigint unsigned)
  - currency_id (varchar(255))
  - year (int)
  - month (int)
  - created_at (timestamp)
  - updated_at (timestamp)


Eliminación de columna que no aportan información relevante (created_at, update_at)
También se eliminan las constantes tales como (currency_id )

In [ ]:
# --- Paso 1: Conectarse a la base de datos y cargar la tabla 'accounting_account_balances' ---
print("Conectando a la base de datos y cargando la tabla 'accounting_account_balances'...")
try:
    engine = get_engine()
    df_account_balances = pd.read_sql('SELECT * FROM accounting_account_balances', engine)
    print(f"Tabla 'accounting_account_balances' cargada exitosamente. Filas: {df_account_balances.shape[0]}, Columnas: {df_account_balances.shape[1]}")
except Exception as e:
    print(f"Error al cargar la tabla 'accounting_account_balances': {e}")
    raise # Relanzar la excepción para detener la ejecución si falla la carga.

# --- Paso 2: Definir y eliminar las columnas ---
print("\nEliminando columnas 'created_at', 'updated_at', y 'currency_id'...")

columns_to_drop = ['created_at', 'updated_at', 'currency_id']

# Usamos .drop() para eliminar las columnas.
# `axis=1` indica que estamos eliminando columnas (no filas).
# `inplace=True` modificaría el DataFrame original directamente.
# Si quieres crear un nuevo DataFrame y mantener el original, omite `inplace=True`
# y asigna el resultado a una nueva variable, por ejemplo:
# df_account_balances_cleaned = df_account_balances.drop(columns=columns_to_drop, axis=1)

# Para este ejemplo, modificaremos el DataFrame original para que sigas trabajando con él
df_account_balances.drop(columns=columns_to_drop, inplace=True)

print(f"Columnas {columns_to_drop} eliminadas exitosamente.")

# --- Paso 3: Verificar las columnas restantes ---
print(f"\nColumnas restantes en el DataFrame 'accounting_account_balances':")
print(df_account_balances.columns.tolist())

# --- Paso 4: Mostrar las primeras filas del DataFrame modificado ---
print("\nPrimeras 5 filas del DataFrame 'accounting_account_balances' después de la eliminación:")
print(df_account_balances.head())

print("\nProceso de eliminación de columnas completado.")

Conectando a la base de datos y cargando la tabla 'accounting_account_balances'...
Tabla 'accounting_account_balances' cargada exitosamente. Filas: 4499, Columnas: 15

Eliminando columnas 'created_at', 'updated_at', y 'currency_id'...
Columnas ['created_at', 'updated_at', 'currency_id'] eliminadas exitosamente.

Columnas restantes en el DataFrame 'accounting_account_balances':
['id', 'code', 'accounting_id', 'name', 'initial_balance', 'final_balance', 'debit_movement', 'credit_movement', 'third_party_type_id', 'third_party_id', 'year', 'month']

Primeras 5 filas del DataFrame 'accounting_account_balances' después de la eliminación:
   id  code  accounting_id    name  initial_balance  final_balance  \
0   1   1.0              1  Activo              0.0  -4.500000e+04   
1   2   1.0              1  Activo              0.0   3.550000e+06   
2   3   1.0              1  Activo              0.0   0.000000e+00   
3   4   1.0              1  Activo              0.0   3.730769e+03   
4   5   1.

Conversión de la tabla a formato csv

In [ ]:
"""# --- Paso 1: Recargar la tabla 'accounting_account_balances' y aplicar las transformaciones ---
# Es CRUCIAL que el DataFrame refleje las últimas operaciones (ej. eliminación de columnas)
# Si tu script anterior ya dejó 'df_account_balances' en el estado deseado, puedes omitir esta recarga y transformación.
# Sin embargo, para asegurar la reproducibilidad y que el CSV contenga el DataFrame limpio,
# es buena práctica recargar y aplicar las transformaciones si este es un script independiente.

print("Preparando el DataFrame 'accounting_account_balances' para exportación...")
try:
    engine = get_engine()
    df_account_balances = pd.read_sql('SELECT * FROM accounting_account_balances', engine)
    
    # Aplicar las eliminaciones de columnas que definimos previamente
    columns_to_drop = ['created_at', 'updated_at', 'currency_id']
    # Filtrar solo las columnas que realmente existen en el DataFrame para evitar errores si ya fueron eliminadas
    existing_columns_to_drop = [col for col in columns_to_drop if col in df_account_balances.columns]
    
    if existing_columns_to_drop:
        df_account_balances.drop(columns=existing_columns_to_drop, inplace=True)
        print(f"Columnas {existing_columns_to_drop} eliminadas antes de exportar.")
    else:
        print("Las columnas a eliminar ('created_at', 'updated_at', 'currency_id') no se encontraron o ya fueron eliminadas.")

    print(f"DataFrame 'accounting_account_balances' listo para exportación. Filas: {df_account_balances.shape[0]}, Columnas: {df_account_balances.shape[1]}")

except Exception as e:
    print(f"Error al preparar el DataFrame 'accounting_account_balances': {e}")
    raise # Relanzar la excepción para detener la ejecución si falla.

# --- Paso 2: Definir la ruta y el nombre del archivo CSV ---
# Puedes ajustar la ruta donde quieres guardar el archivo.
# 'os.getcwd()' te da el directorio de trabajo actual donde se está ejecutando el script.
output_directory = os.getcwd() # Guarda en el mismo directorio del script
csv_filename = 'accounting_account_balances_cleaned.csv'
output_filepath = os.path.join(output_directory, csv_filename)

# --- Paso 3: Exportar el DataFrame a un archivo CSV ---
print(f"\nExportando el DataFrame a CSV: '{output_filepath}'...")
try:
    df_account_balances.to_csv(output_filepath, index=False, encoding='utf-8')
    print("¡DataFrame exportado a CSV exitosamente!")
    print(f"Puedes encontrar el archivo en: {output_filepath}")
except Exception as e:
    print(f"Error al exportar el DataFrame a CSV: {e}")

print("\nProceso de exportación a CSV completado.")"""

'# --- Paso 1: Recargar la tabla \'accounting_account_balances\' y aplicar las transformaciones ---\n# Es CRUCIAL que el DataFrame refleje las últimas operaciones (ej. eliminación de columnas)\n# Si tu script anterior ya dejó \'df_account_balances\' en el estado deseado, puedes omitir esta recarga y transformación.\n# Sin embargo, para asegurar la reproducibilidad y que el CSV contenga el DataFrame limpio,\n# es buena práctica recargar y aplicar las transformaciones si este es un script independiente.\n\nprint("Preparando el DataFrame \'accounting_account_balances\' para exportación...")\ntry:\n    engine = get_engine()\n    df_account_balances = pd.read_sql(\'SELECT * FROM accounting_account_balances\', engine)\n\n    # Aplicar las eliminaciones de columnas que definimos previamente\n    columns_to_drop = [\'created_at\', \'updated_at\', \'currency_id\']\n    # Filtrar solo las columnas que realmente existen en el DataFrame para evitar errores si ya fueron eliminadas\n    existing_col

In [ ]:
# --- Paso 1: Conectarse a la base de datos y cargar la tabla 'accounting_account_balances' ---
# Es CRUCIAL recargar el DataFrame y aplicar las transformaciones para asegurar
# que estamos trabajando con la versión limpia de la tabla.
print("Conectando a la base de datos y cargando la tabla 'accounting_account_balances' para análisis...")
try:
    engine = get_engine()
    df_account_balances = pd.read_sql('SELECT * FROM accounting_account_balances', engine)
    
    # Aplicar las eliminaciones de columnas que definimos previamente
    columns_to_drop = ['created_at', 'updated_at', 'currency_id']
    existing_columns_to_drop = [col for col in columns_to_drop if col in df_account_balances.columns]
    
    if existing_columns_to_drop:
        df_account_balances.drop(columns=existing_columns_to_drop, inplace=True)
        print(f"Columnas {existing_columns_to_drop} eliminadas para el análisis.")
    else:
        print("Las columnas ('created_at', 'updated_at', 'currency_id') no se encontraron o ya fueron eliminadas.")

    print(f"DataFrame 'accounting_account_balances' listo para análisis. Filas: {df_account_balances.shape[0]}, Columnas: {df_account_balances.shape[1]}")

except Exception as e:
    print(f"Error al preparar el DataFrame 'accounting_account_balances': {e}")
    raise # Relanzar la excepción si la carga o limpieza falla.

# --- Paso 2: Análisis Descriptivo de Columnas Numéricas ---
print("\n--- Análisis Descriptivo de Columnas Numéricas ---")
# Seleccionar solo las columnas numéricas (enteros y flotantes)
numeric_cols = df_account_balances.select_dtypes(include=[np.number]).columns.tolist()

if numeric_cols:
    print(df_account_balances[numeric_cols].describe())
else:
    print("No se encontraron columnas numéricas para describir.")

# --- Paso 3: Análisis de Distribución de Columnas Categóricas/Discretas ---
print("\n--- Análisis de Distribución de Columnas Categóricas/Discretas ---")

# Identificar columnas categóricas o discretas con un número manejable de valores únicos
# Excluimos las columnas que son IDs puros o que tienen demasiados valores únicos para un análisis de frecuencia
# Consideramos columnas con <= 50 valores únicos como "categóricas" o "discretas" para este análisis.
# Ajusta este umbral si tienes más o menos categorías esperadas.
for col in df_account_balances.columns:
    if col in numeric_cols and df_account_balances[col].nunique() > 50:
        continue # Saltar IDs o columnas numéricas con muchos valores únicos
    
    if col in ['id', 'accounting_id', 'third_party_id']: # Saltar IDs específicos si no se quiere su frecuencia
        continue
        
    print(f"\nDistribución de la columna '{col}':")
    # Mostrar los top N valores únicos y sus conteos/porcentajes
    # El 10 es un valor arbitrario, puedes ajustarlo.
    value_counts = df_account_balances[col].value_counts(dropna=False) # dropna=False para incluir nulos si los hubiera
    value_percentages = df_account_balances[col].value_counts(dropna=False, normalize=True) * 100

    distribution_df = pd.DataFrame({
        'Count': value_counts,
        'Percentage': value_percentages
    })
    
    print(distribution_df.head(10)) # Mostrar los top 10 valores
    
    if df_account_balances[col].nunique() > 10:
        print(f"... y {df_account_balances[col].nunique() - 10} valores únicos más.")


print("\nProceso de análisis descriptivo y de distribución completado para 'accounting_account_balances'.")

# Opcional: Si quieres guardar este DataFrame limpio y transformado para futuras operaciones
# puedes usar df_account_balances.to_csv('accounting_account_balances_current_state.csv', index=False)

Conectando a la base de datos y cargando la tabla 'accounting_account_balances' para análisis...
Columnas ['created_at', 'updated_at', 'currency_id'] eliminadas para el análisis.
DataFrame 'accounting_account_balances' listo para análisis. Filas: 4499, Columnas: 12

--- Análisis Descriptivo de Columnas Numéricas ---
               id         code  accounting_id  initial_balance  final_balance  \
count  4499.00000  4499.000000    4499.000000     4.499000e+03   4.499000e+03   
mean   2250.00000     2.953210  117162.768615     1.811021e+08   8.990513e+07   
std    1298.89376     1.615093  182842.279222     6.639901e+10   5.615416e+10   
min       1.00000     1.000000       1.000000    -9.999962e+11  -9.999962e+11   
25%    1125.50000     1.408010      24.000000     0.000000e+00   1.424200e+04   
50%    2250.00000     2.501010    2502.000000     0.000000e+00   3.500000e+05   
75%    3374.50000     5.000000  240401.000000     1.606800e+06   1.897309e+06   
max    4499.00000     6.107990  61

**ANALISIS TABLA accounting_account_balances**

1. id
Tipo de Dato Esperado: bigint unsigned (Entero, clave primaria)
Descripción: Identificador único para cada registro de balance de cuenta contable. Es la clave primaria de la tabla.

2. code
Tipo de Dato Esperado: double(16,15) (Numérico, código de cuenta)
Descripción: Código de la cuenta contable al que se refiere este balance. Este código sigue la estructura del plan de cuentas (ej., 110505 para Caja General). Es vital para identificar la cuenta específica.
Relevancia para KPIs: Nivel Alto.
Fundamental para casi cualquier KPI financiero. Permite agrupar y filtrar balances por tipo de cuenta (Activo, Pasivo, Patrimonio, Ingresos, Gastos, Costos).


3. accounting_id
Tipo de Dato Esperado: bigint unsigned (Entero, clave foránea)
Descripción: Identificador único de la cuenta contable maestro a la que pertenece este balance. Es probable que sea una clave foránea a la tabla accounting_accounts (donde id es la clave primaria).
Relevancia para KPIs: Nivel Alto.
Similar a code, pero es la clave numérica para vincular directamente con los metadatos de la cuenta contable (nombre, naturaleza, tipo, etc.) en la tabla accounting_accounts.
Permite desglosar KPIs por la jerarquía del plan de cuentas (clase, grupo, cuenta, subcuenta).

4. name
Tipo de Dato Esperado: varchar(255) (Texto, nombre de la cuenta)
Descripción: Nombre descriptivo de la cuenta contable (ej., "Caja General", "Cuentas por Cobrar Clientes").
Relevancia para KPIs: Nivel Medio-Alto.
No se usa en cálculos, pero es esencial para la legibilidad y la interpretación de los KPIs. Los usuarios finales no verán un code sino el name.
Facilita la creación de informes y dashboards comprensibles.

5. initial_balance
Tipo de Dato Esperado: double(18,6) (Numérico, monetario)
Descripción: Saldo de la cuenta contable al inicio del período (mes y año).
Relevancia para KPIs: Nivel Alto.
Fundamental para calcular el cambio en el saldo durante el período y para verificar la cuadratura contable.
Usado en KPIs de flujos y evolución de cuentas.

6. final_balance
Tipo de Dato Esperado: double(18,6) (Numérico, monetario)
Descripción: Saldo de la cuenta contable al final del período (mes y año).
Relevancia para KPIs: Nivel Muy Alto.
Directamente utilizado para la construcción del Balance General y el Estado de Resultados al final de cada período.
La base para KPIs de liquidez (Cash), solvencia (Deuda), endeudamiento, patrimonio, activos totales, etc.

7. debit_movement
Tipo de Dato Esperado: double(18,6) (Numérico, monetario)
Descripción: Suma total de los movimientos de débito (entradas o aumentos, según la naturaleza de la cuenta) para la cuenta contable durante el período.
Relevancia para KPIs: Nivel Alto.
Permite analizar el flujo bruto de actividad en una cuenta.
Junto con credit_movement, es crucial para verificar la ecuación contable (SaldoInicial+Débitos −Créditos =SaldoFinal) y para el análisis de la actividad transaccional.
Puede usarse para KPIs de volumen de transacciones o actividad por cuenta.

8. credit_movement
Tipo de Dato Esperado: double(18,6) (Numérico, monetario)
Descripción: Suma total de los movimientos de crédito (salidas o disminuciones, según la naturaleza de la cuenta) para la cuenta contable durante el período.
Relevancia para KPIs: Nivel Alto.
Similar a debit_movement, esencial para el análisis del flujo bruto y la validación contable.
Utilizado para KPIs de volumen de transacciones.

9. third_party_type_id
Tipo de Dato Esperado: varchar(50) (Texto, tipo de tercero)
Descripción: Identificador del tipo de tercero asociado a la cuenta (ej., "Cliente", "Proveedor", "Empleado", etc.).
Relevancia para KPIs: Nivel Medio-Alto.
Permite desglosar balances por tipo de contraparte. Por ejemplo, "cuentas por cobrar a clientes" vs "cuentas por cobrar a empleados".
Útil para análisis de riesgo de clientes/proveedores, antigüedad de cartera segmentada por tipo de tercero.

10. third_party_id
Tipo de Dato Esperado: bigint unsigned (Entero, ID de tercero)
Descripción: Identificador único del tercero específico (persona o empresa) asociado a la cuenta. Es probable que sea una clave foránea a una tabla de "terceros".
Relevancia para KPIs: Nivel Alto.
Fundamental para análisis a nivel de cliente/proveedor individual.
KPIs: Concentración de clientes/proveedores, volumen de negocios por tercero, antigüedad de cartera por tercero, riesgo de impago por tercero.

11. year
Tipo de Dato Esperado: int (Entero)
Descripción: Año al que corresponde este balance de cuenta.
Relevancia para KPIs: Nivel Muy Alto.
Esencial para cualquier análisis de tendencia, comparativas anuales, y para la generación de estados financieros por año.
Permite calcular KPIs anuales como el crecimiento de ingresos, evolución de gastos, etc.

12. month
Tipo de Dato Esperado: int (Entero)
Descripción: Mes al que corresponde este balance de cuenta (1 para enero, 12 para diciembre).
Relevancia para KPIs: Nivel Muy Alto.
Esencial para análisis de tendencia mensual, comparativas mensuales, estacionalidad, y para la generación de estados financieros mensuales/trimestrales.
Permite KPIs de liquidez a corto plazo, tendencias de ventas mensuales, etc.

___

***2. Tabla accounting_accounts***

____

In [ ]:
# 1. Reconectar (o asegurar que la conexión esté activa)
try:
    conexion = get_connection()
    cursor = conexion.cursor()

    # 2. Ejecutar la consulta para describir la tabla
    cursor.execute("DESCRIBE accounting_accounts;")
    columns_info = cursor.fetchall()

    print("Columnas de la tabla 'accounting_accounts':")
    for column in columns_info:
        print(f"  - {column[0]} ({column[1]})")

except Exception as e:
    print(f"Ocurrió un error: {e}")

✅ Conexión exitosa a la base de datos
Columnas de la tabla 'accounting_accounts':
  - id (bigint unsigned)
  - name (varchar(250))
  - is_class (tinyint(1))
  - is_group (tinyint(1))
  - is_account (tinyint(1))
  - is_subaccount (tinyint(1))
  - is_auxiliary (tinyint(1))
  - is_subauxiliary (tinyint(1))
  - niif (tinyint(1))
  - cash_flow (tinyint(1))
  - exogenous (tinyint(1))
  - base_value (tinyint(1))
  - nature (varchar(1))
  - term (tinyint(1))
  - favorite (tinyint(1))
  - status (tinyint(1))
  - created_at (timestamp)
  - updated_at (timestamp)


Detección de Columnas Constantes

In [ ]:
# --- Paso 1: Conectarse a la base de datos y cargar la tabla 'accounting_accounts' ---
print("Conectando a la base de datos y cargando la tabla 'accounting_accounts'...")
try:
    engine = get_engine()
    df_accounting_accounts = pd.read_sql('SELECT * FROM accounting_accounts', engine)
    print(f"Tabla 'accounting_accounts' cargada exitosamente. Filas: {df_accounting_accounts.shape[0]}, Columnas: {df_accounting_accounts.shape[1]}")
except Exception as e:
    print(f"Error al cargar la tabla 'accounting_accounts': {e}")
    raise # Relanzar la excepción para detener la ejecución si falla la carga.

# --- Paso 2: Identificar columnas con valores constantes ---
print("\nIdentificando columnas con valores constantes en 'accounting_accounts'...")

constant_columns = []
for col in df_accounting_accounts.columns:
    # `nunique()` cuenta el número de valores únicos en una columna.
    # Si es 1, significa que todos los valores son el mismo (constante).
    if df_accounting_accounts[col].nunique() == 1:
        constant_columns.append(col)

if constant_columns:
    print(f"\nColumnas con valores constantes identificadas en 'accounting_accounts':")
    for col in constant_columns:
        print(f"  - '{col}': Valor único = {df_accounting_accounts[col].iloc[0]}") # Muestra el valor constante
else:
    print("No se encontraron columnas con valores constantes en la tabla 'accounting_accounts'.")

print("\nProceso de identificación de columnas constantes completado para 'accounting_accounts'.")

# --- Opcional: Mostrar las primeras filas del DataFrame para referencia ---
print("\nPrimeras 5 filas del DataFrame 'accounting_accounts':")
print(df_accounting_accounts.head())

Conectando a la base de datos y cargando la tabla 'accounting_accounts'...
Tabla 'accounting_accounts' cargada exitosamente. Filas: 2125, Columnas: 18

Identificando columnas con valores constantes en 'accounting_accounts'...

Columnas con valores constantes identificadas en 'accounting_accounts':
  - 'is_subauxiliary': Valor único = 0
  - 'cash_flow': Valor único = 0
  - 'exogenous': Valor único = 0
  - 'base_value': Valor único = 0
  - 'term': Valor único = 0
  - 'favorite': Valor único = 0
  - 'status': Valor único = 1

Proceso de identificación de columnas constantes completado para 'accounting_accounts'.

Primeras 5 filas del DataFrame 'accounting_accounts':
   id        name  is_class  is_group  is_account  is_subaccount  \
0   1      Activo         1         0           0              0   
1   2     Pasivos         1         0           0              0   
2   3  Patrimonio         1         0           0              0   
3   4    Ingresos         1         0           0       

Eliminación de Columna Constante

In [ ]:
"""
# --- Paso 1: Conectarse a la base de datos y cargar la tabla 'accounting_accounts' ---
print("Conectando a la base de datos y cargando la tabla 'accounting_accounts'...")
try:
    engine = get_engine()
    df_accounting_accounts = pd.read_sql('SELECT * FROM accounting_accounts', engine)
    print(f"Tabla 'accounting_accounts' cargada exitosamente. Filas: {df_accounting_accounts.shape[0]}, Columnas: {df_accounting_accounts.shape[1]}")
except Exception as e:
    print(f"Error al cargar la tabla 'accounting_accounts': {e}")
    raise # Relanzar la excepción para detener la ejecución si falla la carga.

# --- Paso 2: Definir y eliminar las columnas constantes ---
print("\nEliminando columnas constantes de 'accounting_accounts'...")

# Lista de columnas identificadas como constantes
columns_to_drop_constant = [
    'is_subauxiliary',
    'cash_flow',
    'exogenous',
    'base_value',
    'term',
    'favorite',
    'status'
]

# Es buena práctica verificar que las columnas existen antes de intentar eliminarlas
existing_columns_to_drop = [col for col in columns_to_drop_constant if col in df_accounting_accounts.columns]

if existing_columns_to_drop:
    df_accounting_accounts.drop(columns=existing_columns_to_drop, inplace=True)
    print(f"Columnas {existing_columns_to_drop} eliminadas exitosamente.")
else:
    print("Ninguna de las columnas constantes a eliminar se encontró en el DataFrame o ya fueron eliminadas.")

# --- Paso 3: Verificar las columnas restantes ---
print(f"\nColumnas restantes en el DataFrame 'accounting_accounts':")
print(df_accounting_accounts.columns.tolist())

# --- Paso 4: Mostrar las primeras filas del DataFrame modificado ---
print("\nPrimeras 5 filas del DataFrame 'accounting_accounts' después de la eliminación:")
print(df_accounting_accounts.head())

print("\nProceso de eliminación de columnas constantes completado para 'accounting_accounts'.") """

'\n# --- Paso 1: Conectarse a la base de datos y cargar la tabla \'accounting_accounts\' ---\nprint("Conectando a la base de datos y cargando la tabla \'accounting_accounts\'...")\ntry:\n    engine = get_engine()\n    df_accounting_accounts = pd.read_sql(\'SELECT * FROM accounting_accounts\', engine)\n    print(f"Tabla \'accounting_accounts\' cargada exitosamente. Filas: {df_accounting_accounts.shape[0]}, Columnas: {df_accounting_accounts.shape[1]}")\nexcept Exception as e:\n    print(f"Error al cargar la tabla \'accounting_accounts\': {e}")\n    raise # Relanzar la excepción para detener la ejecución si falla la carga.\n\n# --- Paso 2: Definir y eliminar las columnas constantes ---\nprint("\nEliminando columnas constantes de \'accounting_accounts\'...")\n\n# Lista de columnas identificadas como constantes\ncolumns_to_drop_constant = [\n    \'is_subauxiliary\',\n    \'cash_flow\',\n    \'exogenous\',\n    \'base_value\',\n    \'term\',\n    \'favorite\',\n    \'status\'\n]\n\n# Es buen

**ANALISIS accounting_accounts**

In [ ]:
import pandas as pd
import numpy as np # Necesario para np.number
from sqlalchemy import create_engine

# Asumo que 'engine' y 'df_accounting_accounts' ya fueron establecidos
# en los bloques anteriores y están disponibles.
from db_connection import get_engine # Asegúrate de que esta importación esté presente si es necesario.

print("--- Bloque 3.3: Analizando distribución de columnas en 'accounting_accounts' ---")

# En un flujo de script secuencial, df_accounting_accounts ya estaría cargado.
# Si estás ejecutando este bloque de forma independiente, recarga el DataFrame:
if 'df_accounting_accounts' not in locals() or df_accounting_accounts.empty:
    try:
        engine = get_engine()
        df_accounting_accounts = pd.read_sql('SELECT * FROM accounting_accounts', engine)
        print("DataFrame 'accounting_accounts' recargado para el Bloque 3.3.")
    except Exception as e:
        print(f"Error al recargar el DataFrame para el Bloque 3.3: {e}")
        raise

# --- Análisis Descriptivo de Columnas Numéricas ---
print("\n--- Estadísticas Descriptivas para Columnas Numéricas ---")
numeric_cols = df_accounting_accounts.select_dtypes(include=np.number).columns.tolist()

if numeric_cols:
    # Excluimos 'id' de la descripción numérica si es un simple identificador sin significado estadístico
    # Sin embargo, para booleanos (0/1) que son int64, describe() es útil para ver su distribución.
    # Aquí vamos a describir todas las numéricas y luego enfocarnos en las no booleanas para un análisis más profundo.
    
    # Columnas numéricas que NO son IDs puros o flags booleanas constantes (las identificadas en 3.2)
    # Revisa si hay alguna columna numérica que no sea un flag constante 0/1.
    # Del output anterior, todas nuestras 'int64' excepto 'id' y 'niif' parecen ser flags.
    # 'id' y 'niif' también son int64.
    
    # Listamos todas las columnas numéricas
    print("Todas las columnas numéricas:")
    print(df_accounting_accounts[numeric_cols].describe())

    # Consideremos específicamente las columnas numéricas que *podrían* tener rangos y variabilidad
    # Excluyendo 'id' y las que ya sabemos que son constantes de 0/1.
    cols_for_deeper_numeric_analysis = []
    
    # Recargamos la lista de constantes para este bloque si es necesario (para un script independiente)
    constant_columns = []
    for col in df_accounting_accounts.columns:
        if df_accounting_accounts[col].nunique() == 1:
            constant_columns.append(col)
            
    for col in numeric_cols:
        if col not in constant_columns and col != 'id': # Excluir ID y constantes
            cols_for_deeper_numeric_analysis.append(col)

    if cols_for_deeper_numeric_analysis:
        print("\nEstadísticas Descriptivas para Columnas Numéricas con Variabilidad (excluyendo IDs y constantes):")
        print(df_accounting_accounts[cols_for_deeper_numeric_analysis].describe())
    else:
        print("\nNo se encontraron columnas numéricas con variabilidad significativa (más allá de IDs o constantes) para un análisis descriptivo más profundo.")

else:
    print("No se encontraron columnas numéricas en el DataFrame.")


# --- Análisis de Distribución de Columnas Categóricas/Discretas ---
print("\n--- Análisis de Distribución para Columnas Categóricas/Discretas ---")

# Seleccionar columnas de tipo 'object' (cadenas) y también 'int64' que no sean IDs
# y que no sean las columnas constantes que ya identificamos.
categorical_cols = df_accounting_accounts.select_dtypes(include='object').columns.tolist()

# Incluir las columnas numéricas que son flags o tienen pocos valores únicos y no son constantes
# Por ejemplo, 'is_class', 'is_group', 'is_account', 'is_subaccount', 'is_auxiliary', 'niif'
# Aunque 'niif' es int64, tiene 2125 non-null y su valor de '1' del head sugiere que podría ser constante,
# pero su comportamiento de 'flag' la hace relevante para un value_counts.
# Las columnas 'is_subauxiliary', 'cash_flow', 'exogenous', 'base_value', 'term', 'favorite', 'status'
# ya sabemos que son constantes, así que no las incluimos en este análisis de distribución detallado.

# Agregamos las columnas que parecen flags (int64 con pocos valores únicos)
potential_flag_cols = [
    'is_class', 'is_group', 'is_account', 'is_subaccount', 'is_auxiliary', 'niif'
]

# Combinamos y filtramos para no duplicar ni incluir constantes ya conocidas
cols_for_category_analysis = []
cols_for_category_analysis.extend(categorical_cols) # 'name', 'nature'

for col in potential_flag_cols:
    if col not in constant_columns and col not in cols_for_category_analysis:
        cols_for_category_analysis.append(col)

if cols_for_category_analysis:
    for col in cols_for_category_analysis:
        print(f"\nDistribución de la columna '{col}':")
        # Mostrar los top N valores únicos y sus conteos/porcentajes
        # dropna=False para incluir nulos si los hubiera (aunque ya sabemos que no hay en esta tabla)
        value_counts = df_accounting_accounts[col].value_counts(dropna=False)
        value_percentages = df_accounting_accounts[col].value_counts(dropna=False, normalize=True) * 100

        distribution_df = pd.DataFrame({
            'Count': value_counts,
            'Percentage': value_percentages
        })
        
        # Ajustamos para mostrar más si hay muchas categorías o solo las relevantes
        if col == 'name': # Si 'name' es muy variado, solo mostramos el conteo total
            print(f"Total de nombres de cuentas únicos: {df_accounting_accounts['name'].nunique()}")
            print("Mostrando solo los primeros 10 por brevedad para 'name'.")
            print(distribution_df.head(10))
        else: # Para las demás, mostramos hasta 20 o todas si son menos
            print(distribution_df.head(20) if distribution_df.shape[0] > 20 else distribution_df)
else:
    print("No se encontraron columnas categóricas o discretas con variabilidad para analizar.")


print("\n--- Bloque 3.3 Completado ---")

# df_accounting_accounts permanece sin cambios en este bloque.

--- Bloque 3.3: Analizando distribución de columnas en 'accounting_accounts' ---

--- Estadísticas Descriptivas para Columnas Numéricas ---
Todas las columnas numéricas:
                 id     is_class     is_group   is_account  is_subaccount  \
count  2.125000e+03  2125.000000  2125.000000  2125.000000    2125.000000   
mean   2.872738e+05     0.003765     0.019294     0.155765       0.820706   
std    3.114951e+05     0.061256     0.137589     0.362717       0.383689   
min    1.000000e+00     0.000000     0.000000     0.000000       0.000000   
25%    1.307050e+05     0.000000     0.000000     0.000000       1.000000   
50%    2.311160e+05     0.000000     0.000000     0.000000       1.000000   
75%    5.102020e+05     0.000000     0.000000     0.000000       1.000000   
max    1.102010e+07     1.000000     1.000000     1.000000       1.000000   

       is_auxiliary  is_subauxiliary         niif  cash_flow  exogenous  \
count   2125.000000           2125.0  2125.000000     2125.0 

Estadísticas Descriptivas para Columnas Numéricas:

id: Es un identificador único, como se espera. El rango (min 1, max 1.1e+07) es amplio, lo cual es normal para IDs. No se usa en cálculos de KPIs directamente.
Columnas "Flag" (is_class, is_group, is_account, is_subaccount, is_auxiliary, niif):
Todas estas columnas son binarias (0 o 1), lo cual es consistente con su naturaleza de "flags" (indicadores booleanos).
Sus mean (media) reflejan la proporción de 1s en la columna. Por ejemplo:
is_class: 0.37% son clases (valor 1), el resto (0) no lo son.
is_group: 1.93% son grupos.
is_account: 15.58% son cuentas.
is_subaccount: ¡82.07% son subcuentas! Esto indica que la mayoría de los registros en esta tabla son a nivel de subcuenta, lo cual es típico en un plan de cuentas detallado.
is_auxiliary: Solo un 0.047% son auxiliares.
niif: Un 99.95% están bajo NIIF (valor 1), con solo un registro en 0.

Columnas Constantes: Confirmado que is_subauxiliary, cash_flow, exogenous, base_value, term, favorite, status son constantes. Estas son fuertes candidatas para eliminación.
Columnas Casi Constantes: is_auxiliary y niif tienen muy poca variabilidad (un solo valor diferente). Podrían eliminarse si su presencia no es crítica y se busca la máxima simplicidad. Mi recomendación inicial sería eliminarlas también, ya que casi no aportan información.

In [ ]:
# En un flujo de script secuencial, df_accounting_accounts ya estaría cargado.
# Si estás ejecutando este bloque de forma independiente, recarga el DataFrame:
if 'df_accounting_accounts' not in locals() or df_accounting_accounts.empty:
    try:
        engine = get_engine()
        df_accounting_accounts = pd.read_sql('SELECT * FROM accounting_accounts', engine)
        print("DataFrame 'accounting_accounts' recargado para el Bloque 3.4.")
    except Exception as e:
        print(f"Error al recargar el DataFrame para el Bloque 3.4: {e}")
        raise

# Columnas identificadas como constantes o casi constantes:
columns_to_drop = [
    'is_subauxiliary', # Constante (0)
    'cash_flow',       # Constante (0)
    'exogenous',       # Constante (0)
    'base_value',      # Constante (0)
    'term',            # Constante (0)
    'favorite',        # Constante (0)
    'status',          # Constante (1)
    'is_auxiliary',    # Casi constante (99.95% en 0)
    'niif',            # Casi constante (99.95% en 1)
    'created_at',      # Metadato de fecha/hora de creación
    'updated_at'       # Metadato de fecha/hora de última actualización
]

# Filtramos las columnas que realmente existen en el DataFrame antes de intentar eliminarlas
existing_columns_to_drop = [col for col in columns_to_drop if col in df_accounting_accounts.columns]

if existing_columns_to_drop:
    df_accounting_accounts.drop(columns=existing_columns_to_drop, inplace=True)
    print(f"Columnas eliminadas exitosamente: {existing_columns_to_drop}")
else:
    print("Ninguna de las columnas especificadas para eliminar se encontró en el DataFrame o ya fueron eliminadas.")

# Mostrar las dimensiones del DataFrame después de la eliminación
print(f"\nNuevas dimensiones de 'accounting_accounts': {df_accounting_accounts.shape[0]} filas, {df_accounting_accounts.shape[1]} columnas.")

# Mostrar las primeras filas del DataFrame modificado para verificar
print("\nPrimeras 5 filas del DataFrame 'accounting_accounts' después de la eliminación de columnas:")
print(df_accounting_accounts.head())



# 📊 Análisis Detallado de Columnas en `accounting_accounts` 📊

Este análisis se basa en el output original del Bloque 3.1 y 3.3, previo a cualquier eliminación de columnas, para justificar y documentar las decisiones de limpieza.

---

## **Contexto General de la Tabla `accounting_accounts`**

Esta tabla representa el **Plan de Cuentas Contable** de una empresa. Cada fila describe una cuenta contable específica con sus características y atributos. La **jerarquía contable** (Clase, Grupo, Cuenta, Subcuenta, Auxiliar, Subauxiliar) es fundamental en este tipo de tablas.

En contabilidad, un plan de cuentas es una lista estructurada de todas las cuentas que una organización utiliza para registrar sus transacciones financieras. La estructura jerárquica permite un nivel de detalle creciente, desde categorías generales (Clase) hasta cuentas muy específicas (Subauxiliar).

---

## **Columnas Presentes en el DataFrame Original (Antes de la Limpieza):**

### **Columnas Conservadas (Relevantes para Análisis/Modelado):**

* **`id`**
    * **Tipo de Dato:** `int64`
    * **Descripción:** Identificador único para cada cuenta contable (clave primaria).
    * **Relevancia:** Esencial para la integridad de la base de datos y la vinculación con otras tablas (ej., `accounting_account_balances`).
    * **Decisión:** **Conservada.**

* **`name`**
    * **Tipo de Dato:** `object` (cadena de texto)
    * **Descripción:** Nombre descriptivo de la cuenta contable (ej., "Activo", "Caja General").
    * **Relevancia:** Altamente relevante. Fundamental para la interpretación de cualquier análisis y para la comprensión por parte del usuario. Hay 1205 nombres únicos.
    * **Decisión:** **Conservada.**

* **`is_class`**
    * **Tipo de Dato:** `int64` (flag booleana)
    * **Descripción:** Indica si la cuenta es una "Clase" contable (ej., "Activo", "Pasivo").
    * **Relevancia:** Relevante. Permite categorizar al nivel más alto de la jerarquía contable, crucial para estados financieros. La media (0.003765) indica solo 8 de 2125 son clases, lo cual es correcto.
    * **Decisión:** **Conservada.**

* **`is_group`**
    * **Tipo de Dato:** `int64` (flag booleana)
    * **Descripción:** Indica si la cuenta es un "Grupo" contable (ej., "Disponible", "Inversiones").
    * **Relevancia:** Relevante. Permite agrupaciones importantes dentro de las clases. La media (0.019294) indica 41 de 2125 son grupos.
    * **Decisión:** **Conservada.**

* **`is_account`**
    * **Tipo de Dato:** `int64` (flag booleana)
    * **Descripción:** Indica si la cuenta es una "Cuenta" contable (ej., "Caja", "Bancos").
    * **Relevancia:** Relevante. Nivel de detalle intermedio, útil para análisis por tipo de cuenta. La media (0.155765) indica 331 de 2125 son cuentas.
    * **Decisión:** **Conservada.**

* **`is_subaccount`**
    * **Tipo de Dato:** `int64` (flag booleana)
    * **Descripción:** Indica si la cuenta es una "Subcuenta" contable (ej., "Caja General", "Caja Menor").
    * **Relevancia:** Muy relevante. La mayoría de los registros (82.07%) son subcuentas, nivel donde se registran la mayoría de las transacciones, fundamental para análisis granular.
    * **Decisión:** **Conservada.**

* **`nature`**
    * **Tipo de Dato:** `object` (cadena de texto)
    * **Descripción:** Indica la naturaleza de la cuenta: **D** (Débito) o **C** (Crédito).
    * **Relevancia:** Altamente relevante. Fundamental para entender el comportamiento de los saldos (aumenta con débitos/créditos) y para la validación contable. La distribución (62.26% D, 37.74% C) es esperada.
    * **Decisión:** **Conservada.**

### **Columnas Eliminadas (Baja o Nula Relevancia para Análisis/Modelado):**

* **`is_auxiliary`**
    * **Tipo de Dato:** `int64` (flag booleana)
    * **Descripción:** Indica si la cuenta es una "Auxiliar" contable.
    * **Relevancia:** Baja. El 99.95% de los valores son 0 (solo 1 registro es 1). Aporta variabilidad insignificante.
    * **Decisión:** **Eliminada.**

* **`is_subauxiliary`**
    * **Tipo de Dato:** `int64` (flag booleana)
    * **Descripción:** Indica si la cuenta es una "Subauxiliar" contable.
    * **Relevancia:** Nula. El 100% de los valores son 0. Es una columna constante.
    * **Decisión:** **Eliminada.**

* **`niif`**
    * **Tipo de Dato:** `int64` (flag booleana)
    * **Descripción:** Indica si la cuenta aplica bajo Normas Internacionales de Información Financiera (NIIF).
    * **Relevancia:** Baja. El 99.95% de los valores son 1 (solo 1 registro es 0). No permite una diferenciación útil.
    * **Decisión:** **Eliminada.**

* **`cash_flow`**
    * **Tipo de Dato:** `int64` (flag booleana)
    * **Descripción:** Indica si la cuenta está relacionada con el flujo de efectivo.
    * **Relevancia:** Nula. El 100% de los valores son 0. Es una columna constante.
    * **Decisión:** **Eliminada.**

* **`exogenous`**
    * **Tipo de Dato:** `int64` (flag booleana)
    * **Descripción:** Indica si la cuenta es "exógena" (relacionada con informes fiscales o regulatorios).
    * **Relevancia:** Nula. El 100% de los valores son 0. Es una columna constante.
    * **Decisión:** **Eliminada.**

* **`base_value`**
    * **Tipo de Dato:** `int64`
    * **Descripción:** Posiblemente un valor base o flag para cálculos específicos.
    * **Relevancia:** Nula. El 100% de los valores son 0. Es una columna constante.
    * **Decisión:** **Eliminada.**

* **`term`**
    * **Tipo de Dato:** `int64`
    * **Descripción:** Posiblemente indica el término de la cuenta (corto/largo plazo).
    * **Relevancia:** Nula. El 100% de los valores son 0. Es una columna constante.
    * **Decisión:** **Eliminada.**

* **`favorite`**
    * **Tipo de Dato:** `int64` (flag booleana)
    * **Descripción:** Indicador para marcar cuentas "favoritas" en el sistema.
    * **Relevancia:** Nula. El 100% de los valores son 0. Es una característica interna del sistema, no relevante para el análisis.
    * **Decisión:** **Eliminada.**

* **`status`**
    * **Tipo de Dato:** `int64` (flag booleana)
    * **Descripción:** Posiblemente el estado de la cuenta (activa/inactiva).
    * **Relevancia:** Nula. El 100% de los valores son 1. Es una columna constante; no proporciona información diferencial.
    * **Decisión:** **Eliminada.**

* **`created_at`**
    * **Tipo de Dato:** `datetime64[ns]`
    * **Descripción:** Fecha y hora de creación del registro.
    * **Relevancia:** Baja. Metadato de auditoría. No contribuye directamente a los KPIs financieros o modelos de ML.
    * **Decisión:** **Eliminada.**

* **`updated_at`**
    * **Tipo de Dato:** `datetime64[ns]`
    * **Descripción:** Fecha y hora de la última actualización del registro.
    * **Relevancia:** Baja. Similar a `created_at`, es un metadato.
    * **Decisión:** **Eliminada.**

***3. Tabla accounting_movements***

In [8]:

# 1. Reconectar (o asegurar que la conexión esté activa)
try:
    conexion = get_connection()
    cursor = conexion.cursor()

    # 2. Ejecutar la consulta para describir la tabla
    cursor.execute("DESCRIBE accounting_movements;")
    columns_info = cursor.fetchall()

    print("Columnas de la tabla 'accounting_movements':")
    for column in columns_info:
        print(f"  - {column[0]} ({column[1]})")

except Exception as e:
    print(f"Ocurrió un error: {e}")

Ocurrió un error: name 'get_connection' is not defined


#Se identifica que hay datos faltantes en las columnas: o	references_document_id: 6019 nulos (85.74%)
o	item_id: 5511 nulos (78.50%)
o	item_type: 5511 nulos (78.50%)
o	document: 4510 nulos (64.25%)
o	payroll_employee_reference_id: 2510 nulos (35.75%)

-> Se identifica que las columnas item_type, item_id y references_document_id refieren a lo mismo, por lo cual, solo se llenaba una de las 3, siendo necesario combinar toda la información en una nueva columna
 

In [ ]:
# --- Paso 1: Conectarse a la base de datos y cargar la tabla 'accounting_movements' ---
# *** IMPORTANTE: Si ya ejecutaste el código anterior, asegúrate de recargar el DataFrame ***
print("Conectando a la base de datos y cargando la tabla 'accounting_movements'...")
try:
    engine = get_engine()
    df_accounting_movements = pd.read_sql('SELECT * FROM accounting_movements', engine)
    print(f"Tabla 'accounting_movements' cargada exitosamente. Filas: {df_accounting_movements.shape[0]}, Columnas: {df_accounting_movements.shape[1]}")
except Exception as e:
    print(f"Error al cargar la tabla 'accounting_movements': {e}")
    raise # Relanzar la excepción para detener la ejecución si falla la carga.

# --- Paso 2: Crear las nuevas columnas combinadas ---
print("\nCombinando columnas 'item_id', 'references_document_id', y 'payroll_employee_reference_id'...")

# Inicializar las nuevas columnas con valores nulos
df_accounting_movements['primary_reference_type'] = np.nan
df_accounting_movements['primary_reference_id'] = np.nan

# Lógica de combinación con prioridad:
# 1. payroll_employee_reference_id (más específico)
# 2. references_document_id
# 3. item_id

# Condición 1: Cuando payroll_employee_reference_id tiene un valor
cond_payroll = df_accounting_movements['payroll_employee_reference_id'].notna()
df_accounting_movements.loc[cond_payroll, 'primary_reference_type'] = 'Payroll Employee'
df_accounting_movements.loc[cond_payroll, 'primary_reference_id'] = df_accounting_movements['payroll_employee_reference_id']

# Condición 2: Cuando payroll_employee_reference_id es nulo, pero references_document_id no lo es
cond_doc = (~cond_payroll) & (df_accounting_movements['references_document_id'].notna())
df_accounting_movements.loc[cond_doc, 'primary_reference_type'] = 'Document'
df_accounting_movements.loc[cond_doc, 'primary_reference_id'] = df_accounting_movements['references_document_id']

# Condición 3: Cuando payroll_employee_reference_id y references_document_id son nulos, pero item_id no lo es
cond_item = (~cond_payroll) & (~cond_doc) & (df_accounting_movements['item_id'].notna())
df_accounting_movements.loc[cond_item, 'primary_reference_type'] = 'Item'
df_accounting_movements.loc[cond_item, 'primary_reference_id'] = df_accounting_movements['item_id']

# Para los casos donde ninguna de las referencias se llenó, primary_reference_type e _id quedarán como NaN.
# Rellenar 'primary_reference_type' con 'N/A' para mayor claridad
df_accounting_movements['primary_reference_type'] = df_accounting_movements['primary_reference_type'].fillna('N/A')

print("Columnas combinadas 'primary_reference_type' y 'primary_reference_id' creadas.")

# --- Paso 3: Opcional - Eliminar las columnas originales si ya no son necesarias ---
# Asegúrate de que las columnas 'item_type' y 'document' (de las que hablamos en la revisión anterior)
# también se incluyan aquí si no las necesitas más, ya que también tenían muchos nulos.
# Aquí solo estoy eliminando las que pediste combinar.
columns_to_drop = ['item_id', 'references_document_id', 'payroll_employee_reference_id']
# Si también quieres eliminar 'item_type' y 'document' (de la discusión previa), descomenta la línea de abajo:
# columns_to_drop.extend(['item_type', 'document'])

df_accounting_movements_cleaned = df_accounting_movements.drop(columns=columns_to_drop)
print(f"Columnas originales {columns_to_drop} eliminadas.")

# --- Paso 4: Mostrar las primeras filas con las nuevas columnas ---
print("\nPrimeras 5 filas del DataFrame con las nuevas columnas de referencia:")
print(df_accounting_movements_cleaned[['primary_reference_type', 'primary_reference_id']].head())

# --- Paso 5: Verificar el conteo de nulos para las nuevas columnas ---
print("\nConteo de nulos para las nuevas columnas de referencia:")
print(df_accounting_movements_cleaned[['primary_reference_type', 'primary_reference_id']].isnull().sum())

print("\nProceso de combinación de referencias completado.")

Conectando a la base de datos y cargando la tabla 'accounting_movements'...
Tabla 'accounting_movements' cargada exitosamente. Filas: 7020, Columnas: 22

Combinando columnas 'item_id', 'references_document_id', y 'payroll_employee_reference_id'...
Columnas combinadas 'primary_reference_type' y 'primary_reference_id' creadas.
Columnas originales ['item_id', 'references_document_id', 'payroll_employee_reference_id'] eliminadas.

Primeras 5 filas del DataFrame con las nuevas columnas de referencia:
  primary_reference_type  primary_reference_id
0                   Item                 394.0
1                   Item                 394.0
2                   Item                 394.0
3                   Item                 394.0
4                   Item                 394.0

Conteo de nulos para las nuevas columnas de referencia:
primary_reference_type    0
primary_reference_id      0
dtype: int64

Proceso de combinación de referencias completado.


C:\Users\DUVAN\AppData\Local\Temp\ipykernel_14852\1707058357.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Payroll Employee' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_accounting_movements.loc[cond_payroll, 'primary_reference_type'] = 'Payroll Employee'


In [ ]:
# Asegúrate de que get_engine() está definida en db_connection.py
from db_connection import get_engine

# --- Paso 1: Conectarse a la base de datos y cargar la tabla 'accounting_movements' ---
# *** IMPORTANTE: Si ya ejecutaste el código anterior, asegúrate de recargar el DataFrame ***
print("Conectando a la base de datos y cargando la tabla 'accounting_movements'...")
try:
    engine = get_engine()
    df_accounting_movements = pd.read_sql('SELECT * FROM accounting_movements', engine)
    print(f"Tabla 'accounting_movements' cargada exitosamente. Filas: {df_accounting_movements.shape[0]}, Columnas: {df_accounting_movements.shape[1]}")
except Exception as e:
    print(f"Error al cargar la tabla 'accounting_movements': {e}")
    raise # Relanzar la excepción para detener la ejecución si falla la carga.

# --- Paso 2: Crear las nuevas columnas combinadas ---
print("\nCombinando columnas 'item_id', 'references_document_id', y 'payroll_employee_reference_id'...")

# Inicializar las nuevas columnas con valores nulos
df_accounting_movements['primary_reference_type'] = np.nan
df_accounting_movements['primary_reference_id'] = np.nan

# Lógica de combinación con prioridad:
# 1. payroll_employee_reference_id (más específico)
# 2. references_document_id
# 3. item_id

# Condición 1: Cuando payroll_employee_reference_id tiene un valor
cond_payroll = df_accounting_movements['payroll_employee_reference_id'].notna()
df_accounting_movements.loc[cond_payroll, 'primary_reference_type'] = 'Payroll Employee'
df_accounting_movements.loc[cond_payroll, 'primary_reference_id'] = df_accounting_movements['payroll_employee_reference_id']

# Condición 2: Cuando payroll_employee_reference_id es nulo, pero references_document_id no lo es
cond_doc = (~cond_payroll) & (df_accounting_movements['references_document_id'].notna())
df_accounting_movements.loc[cond_doc, 'primary_reference_type'] = 'Document'
df_accounting_movements.loc[cond_doc, 'primary_reference_id'] = df_accounting_movements['references_document_id']

# Condición 3: Cuando payroll_employee_reference_id y references_document_id son nulos, pero item_id no lo es
cond_item = (~cond_payroll) & (~cond_doc) & (df_accounting_movements['item_id'].notna())
df_accounting_movements.loc[cond_item, 'primary_reference_type'] = 'Item'
df_accounting_movements.loc[cond_item, 'primary_reference_id'] = df_accounting_movements['item_id']

# Para los casos donde ninguna de las referencias se llenó, primary_reference_type e _id quedarán como NaN.
# Rellenar 'primary_reference_type' con 'N/A' para mayor claridad
df_accounting_movements['primary_reference_type'] = df_accounting_movements['primary_reference_type'].fillna('N/A')

print("Columnas combinadas 'primary_reference_type' y 'primary_reference_id' creadas.")

# --- Paso 3: Opcional - Eliminar las columnas originales si ya no son necesarias ---
# *** Hemos añadido 'item_type' a la lista de columnas a eliminar ***
columns_to_drop = ['item_id', 'references_document_id', 'payroll_employee_reference_id', 'item_type']

df_accounting_movements_cleaned = df_accounting_movements.drop(columns=columns_to_drop)
print(f"Columnas originales {columns_to_drop} eliminadas.")

# --- Paso 4: Mostrar las primeras filas con las nuevas columnas ---
print("\nPrimeras 5 filas del DataFrame con las nuevas columnas de referencia:")
# Ajusta las columnas a mostrar en head() si quieres ver otras además de las nuevas de referencia
print(df_accounting_movements_cleaned[['id', 'primary_reference_type', 'primary_reference_id']].head())

# --- Paso 5: Verificar el conteo de nulos para las nuevas columnas ---
print("\nConteo de nulos para las nuevas columnas de referencia:")
print(df_accounting_movements_cleaned[['primary_reference_type', 'primary_reference_id']].isnull().sum())

# --- Paso 6: Verificar que item_type ha sido eliminada ---
print(f"\nColumnas restantes en el DataFrame después de la limpieza: {df_accounting_movements_cleaned.columns.tolist()}")

print("\nProceso de combinación de referencias y eliminación de 'item_type' completado.")

Conectando a la base de datos y cargando la tabla 'accounting_movements'...
Tabla 'accounting_movements' cargada exitosamente. Filas: 7020, Columnas: 22

Combinando columnas 'item_id', 'references_document_id', y 'payroll_employee_reference_id'...
Columnas combinadas 'primary_reference_type' y 'primary_reference_id' creadas.
Columnas originales ['item_id', 'references_document_id', 'payroll_employee_reference_id', 'item_type'] eliminadas.

Primeras 5 filas del DataFrame con las nuevas columnas de referencia:
   id primary_reference_type  primary_reference_id
0  74                   Item                 394.0
1  75                   Item                 394.0
2  76                   Item                 394.0
3  77                   Item                 394.0
4  78                   Item                 394.0

Conteo de nulos para las nuevas columnas de referencia:
primary_reference_type    0
primary_reference_id      0
dtype: int64

Columnas restantes en el DataFrame después de la limp

C:\Users\DUVAN\AppData\Local\Temp\ipykernel_14852\289087782.py:29: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Payroll Employee' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_accounting_movements.loc[cond_payroll, 'primary_reference_type'] = 'Payroll Employee'


----

***4.Tabla accounting_voucher_items***

---

In [ ]:
# 1. Reconectar (o asegurar que la conexión esté activa)
try:
    conexion = get_connection()
    cursor = conexion.cursor()

    # 2. Ejecutar la consulta para describir la tabla
    cursor.execute("DESCRIBE accounting_voucher_items;")
    columns_info = cursor.fetchall()

    print("Columnas de la tabla 'accounting_voucher_items':")
    for column in columns_info:
        print(f"  - {column[0]} ({column[1]})")

except Exception as e:
    print(f"Ocurrió un error: {e}")
finally:
    # 3. Cerrar el cursor y la conexión de forma segura
    if 'cursor' in locals() and cursor is not None:
        cursor.close()
    if 'conexion' in locals() and conexion.is_connected():
        conexion.close()
    print("Conexión y cursor cerrados.")

✅ Conexión exitosa a la base de datos
Columnas de la tabla 'accounting_voucher_items':
  - id (bigint unsigned)
  - accounting_voucher_id (bigint unsigned)
  - third_party_type_id (varchar(255))
  - third_party_id (bigint unsigned)
  - accounting_account_id (bigint unsigned)
  - description (varchar(255))
  - debit_movement (double(18,6))
  - credit_movement (double(18,6))
  - created_at (timestamp)
  - updated_at (timestamp)
Conexión y cursor cerrados.


---

***5.Tabla accounting_vouchers***

---

In [ ]:
# 1. Reconectar (o asegurar que la conexión esté activa)
try:
    conexion = get_connection()
    cursor = conexion.cursor()

    # 2. Ejecutar la consulta para describir la tabla
    cursor.execute("DESCRIBE accounting_vouchers;")
    columns_info = cursor.fetchall()

    print("Columnas de la tabla 'accounting_vouchers':")
    for column in columns_info:
        print(f"  - {column[0]} ({column[1]})")

except Exception as e:
    print(f"Ocurrió un error: {e}")
finally:
    # 3. Cerrar el cursor y la conexión de forma segura
    if 'cursor' in locals() and cursor is not None:
        cursor.close()
    if 'conexion' in locals() and conexion.is_connected():
        conexion.close()
    print("Conexión y cursor cerrados.")

✅ Conexión exitosa a la base de datos
Columnas de la tabla 'accounting_vouchers':
  - id (bigint unsigned)
  - accounting_voucher_type_id (bigint unsigned)
  - prefix (varchar(50))
  - number (bigint)
  - voucher_date (date)
  - third_party_id (bigint unsigned)
  - third_party_type_id (varchar(50))
  - advance_account_payable_id (bigint unsigned)
  - supplier_account_id (bigint unsigned)
  - advance_account_receivable_id (bigint unsigned)
  - cartera_account_id (bigint unsigned)
  - currency_id (varchar(255))
  - total_value (double(18,6))
  - voucher_status_id (smallint unsigned)
  - preferred (tinyint(1))
  - created_at (timestamp)
  - updated_at (timestamp)
Conexión y cursor cerrados.


---

***6.Tabla accounting_voucher_types***

---

In [ ]:
# 1. Reconectar (o asegurar que la conexión esté activa)
try:
    conexion = get_connection()
    cursor = conexion.cursor()

    # 2. Ejecutar la consulta para describir la tabla
    cursor.execute("DESCRIBE accounting_voucher_types;")
    columns_info = cursor.fetchall()

    print("Columnas de la tabla 'accounting_voucher_types':")
    for column in columns_info:
        print(f"  - {column[0]} ({column[1]})")

except Exception as e:
    print(f"Ocurrió un error: {e}")
finally:
    # 3. Cerrar el cursor y la conexión de forma segura
    if 'cursor' in locals() and cursor is not None:
        cursor.close()
    if 'conexion' in locals() and conexion.is_connected():
        conexion.close()
    print("Conexión y cursor cerrados.")

✅ Conexión exitosa a la base de datos
Columnas de la tabla 'accounting_voucher_types':
  - id (bigint unsigned)
  - code (varchar(50))
  - name (varchar(60))
  - prefix (varchar(50))
  - initial_number (bigint)
  - current_number (bigint)
  - ledger_id (int unsigned)
  - status (tinyint(1))
  - created_at (timestamp)
  - updated_at (timestamp)
Conexión y cursor cerrados.


---

***7.Tabla retention_concepts***

---

In [ ]:
# 1. Reconectar (o asegurar que la conexión esté activa)
try:
    conexion = get_connection()
    cursor = conexion.cursor()

    # 2. Ejecutar la consulta para describir la tabla
    cursor.execute("DESCRIBE retention_concepts;")
    columns_info = cursor.fetchall()

    print("Columnas de la tabla 'retention_concepts':")
    for column in columns_info:
        print(f"  - {column[0]} ({column[1]})")

except Exception as e:
    print(f"Ocurrió un error: {e}")


✅ Conexión exitosa a la base de datos
Columnas de la tabla 'retention_concepts':
  - id (int unsigned)
  - description (text)
  - account_id (bigint unsigned)
  - created_at (timestamp)
  - updated_at (timestamp)


---

***8.Tabla retentions***

---

In [ ]:
# 1. Reconectar (o asegurar que la conexión esté activa)
try:
    conexion = get_connection()
    cursor = conexion.cursor()

    # 2. Ejecutar la consulta para describir la tabla
    cursor.execute("DESCRIBE retentions;")
    columns_info = cursor.fetchall()

    print("Columnas de la tabla 'retentions':")
    for column in columns_info:
        print(f"  - {column[0]} ({column[1]})")

except Exception as e:
    print(f"Ocurrió un error: {e}")


✅ Conexión exitosa a la base de datos
Columnas de la tabla 'retentions':
  - id (int unsigned)
  - retention_type_id (varchar(255))
  - retention_concept (int unsigned)
  - description (varchar(255))
  - percentage (double(10,5))
  - uvt_base (double(18,6))
  - base (double(18,6))
  - notes (varchar(255))
  - status (tinyint(1))
  - account_id (bigint unsigned)
  - user_id (int unsigned)
  - is_custom (varchar(255))
  - created_at (timestamp)
  - updated_at (timestamp)


---

***9.Tabla retentions_applied***

---

In [ ]:
# 1. Reconectar (o asegurar que la conexión esté activa)
try:
    conexion = get_connection()
    cursor = conexion.cursor()

    # 2. Ejecutar la consulta para describir la tabla
    cursor.execute("DESCRIBE retentions_applied;")
    columns_info = cursor.fetchall()

    print("Columnas de la tabla 'retentions_applied':")
    for column in columns_info:
        print(f"  - {column[0]} ({column[1]})")

except Exception as e:
    print(f"Ocurrió un error: {e}")


✅ Conexión exitosa a la base de datos
Columnas de la tabla 'retentions_applied':
  - id (bigint unsigned)
  - name (varchar(255))
  - type (varchar(255))
  - percentage (double(18,6))
  - base (double(18,6))
  - value (double(18,6))
  - retention_id (int unsigned)
  - contact_id (int unsigned)
  - document_id (bigint unsigned)
  - created_at (timestamp)
  - updated_at (timestamp)


---

***10.Tabla retentions_certificates***

---

In [ ]:
# 1. Reconectar (o asegurar que la conexión esté activa)
try:
    conexion = get_connection()
    cursor = conexion.cursor()

    # 2. Ejecutar la consulta para describir la tabla
    cursor.execute("DESCRIBE retentions_certificates;")
    columns_info = cursor.fetchall()

    print("Columnas de la tabla 'retentions_certificates':")
    for column in columns_info:
        print(f"  - {column[0]} ({column[1]})")

except Exception as e:
    print(f"Ocurrió un error: {e}")


✅ Conexión exitosa a la base de datos
Columnas de la tabla 'retentions_certificates':
  - id (bigint unsigned)
  - contact_id (int unsigned)
  - document_id (bigint unsigned)
  - value (double(18,6))
  - sent (tinyint(1))
  - certificate_url (varchar(255))
  - created_at (timestamp)
  - updated_at (timestamp)


---

***11.Tabla taxes***

---

In [ ]:
# 1. Reconectar (o asegurar que la conexión esté activa)
try:
    conexion = get_connection()
    cursor = conexion.cursor()

    # 2. Ejecutar la consulta para describir la tabla
    cursor.execute("DESCRIBE taxes;")
    columns_info = cursor.fetchall()

    print("Columnas de la tabla 'taxes':")
    for column in columns_info:
        print(f"  - {column[0]} ({column[1]})")

except Exception as e:
    print(f"Ocurrió un error: {e}")


✅ Conexión exitosa a la base de datos
Columnas de la tabla 'taxes':
  - id (smallint unsigned)
  - tax_type_id (varchar(255))
  - description (varchar(255))
  - percentage (double(10,5))
  - notes (varchar(255))
  - status (tinyint(1))
  - tax_account_deductible_id (bigint unsigned)
  - tax_account_generated_id (bigint unsigned)
  - user_id (int unsigned)
  - created_at (timestamp)
  - updated_at (timestamp)


------

***12.Tabla payments***

---

In [ ]:
# 1. Reconectar (o asegurar que la conexión esté activa)
try:
    conexion = get_connection()
    cursor = conexion.cursor()

    # 2. Ejecutar la consulta para describir la tabla
    cursor.execute("DESCRIBE payments;")
    columns_info = cursor.fetchall()

    print("Columnas de la tabla 'payments':")
    for column in columns_info:
        print(f"  - {column[0]} ({column[1]})")

except Exception as e:
    print(f"Ocurrió un error: {e}")
finally:
    # 3. Cerrar el cursor y la conexión de forma segura
    if 'cursor' in locals() and cursor is not None:
        cursor.close()
    if 'conexion' in locals() and conexion.is_connected():
        conexion.close()
    print("Conexión y cursor cerrados.")

✅ Conexión exitosa a la base de datos
Columnas de la tabla 'payments':
  - id (bigint unsigned)
  - document_type_id (int unsigned)
  - billing_numbering_id (int unsigned)
  - payment_type (smallint unsigned)
  - contact_id (int unsigned)
  - number (int unsigned)
  - document_date (date)
  - document_hour (time)
  - associate_invoice (tinyint(1))
  - reference_document (json)
  - advance (tinyint(1))
  - subtotal (double(16,2))
  - discount (double(16,2))
  - gross (double(16,2))
  - pending (double(16,2))
  - total (double(16,2))
  - taxes (json)
  - retentions_value (double(16,2))
  - retentions (json)
  - retentions_details (json)
  - payments (json)
  - notes (varchar(255))
  - comments (varchar(255))
  - document_status_id (smallint unsigned)
  - DIAN_status (json)
  - contact_data (json)
  - details (json)
  - extra_data (json)
  - user_id (int unsigned)
  - deleted_at (timestamp)
  - created_at (timestamp)
  - updated_at (timestamp)
Conexión y cursor cerrados.


In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import json # Necesario para parsear strings JSON si no son objetos nativos

# Asegúrate de que get_engine() está definida en db_connection.py
from db_connection import get_engine

# --- Paso 1: Conectarse a la base de datos y cargar la tabla 'payments' ---
print("Conectando a la base de datos y cargando la tabla 'payments'...")
try:
    engine = get_engine()
    df_payments = pd.read_sql('SELECT * FROM payments', engine)
    print(f"Tabla 'payments' cargada exitosamente. Filas: {df_payments.shape[0]}, Columnas: {df_payments.shape[1]}")
except Exception as e:
    print(f"Error al cargar la tabla 'payments': {e}")
    # Considera salir o manejar el error adecuadamente si la carga falla
    raise # Relanzar la excepción para detener la ejecución si falla la carga.

# --- Paso 2: Identificar columnas JSON y aplanarlas ---
# Definimos las columnas que identificamos como JSON
json_columns = [
    'reference_document',
    'taxes',
    'retentions',
    'retentions_details',
    'payments', # Ojo: esta es la columna 'payments' en la tabla 'payments'.
    'DIAN_status',
    'contact_data',
    'details',
    'extra_data'
]

# Crear una copia del DataFrame para trabajar y mantener el original si es necesario
df_payments_flattened = df_payments.copy()

print("\nIniciando aplanamiento de columnas JSON...")

for col in json_columns:
    if col in df_payments_flattened.columns:
        print(f"Aplanando columna: '{col}'")
        
        # Convertir strings JSON a objetos Python (si no lo son ya)
        # Usamos .astype(str) para asegurar que el contenido sea string antes de json.loads,
        # lo que puede ayudar si hay valores no-string (ej., números)
        df_payments_flattened[col] = df_payments_flattened[col].astype(str).apply(
            lambda x: json.loads(x) if isinstance(x, str) and x.strip() and x.startswith(('[', '{')) else None
        )

        # Filtrar solo las filas donde el JSON no es nulo y es un diccionario/lista
        valid_json_data = df_payments_flattened[df_payments_flattened[col].apply(lambda x: isinstance(x, (dict, list)) and x is not None)][col]
        
        # Obtener los índices de las filas con JSON válido para unir correctamente
        valid_indices = valid_json_data.index

        if not valid_json_data.empty:
            try:
                # Comprobar si el JSON es una lista (para aplanamiento de múltiples registros por ID)
                # O si es un diccionario (para aplanamiento de claves a columnas)
                
                # Para el caso de 'payments' (la columna, no la tabla) o cualquier otra que sea una lista de objetos:
                if valid_json_data.apply(lambda x: isinstance(x, list)).any():
                    # Explode la lista de JSONs en múltiples filas, manteniendo el ID original.
                    # Esto es útil si un solo registro de pago tiene múltiples sub-pagos o ítems.
                    # Crea un DataFrame temporal para la columna JSON aplanada
                    temp_df = pd.json_normalize(valid_json_data.explode())
                    temp_df.index = valid_json_data.explode().index # Asegurar que el índice coincida
                    
                    # Unir con el DataFrame principal. Añadir un prefijo claro.
                    df_payments_flattened = df_payments_flattened.merge(
                        temp_df.add_prefix(f'{col}_'),
                        left_index=True,
                        right_index=True,
                        how='left',
                        suffixes=('', f'_{col}_dup') # Sufijo para columnas duplicadas, si las hay
                    )
                else: # Asumir que es un diccionario o un solo objeto JSON
                    # Aplanar el diccionario.
                    # Se usa `record_path=None` si el JSON de la columna es un diccionario con las claves a aplanar.
                    df_col_flattened = pd.json_normalize(valid_json_data)
                    df_col_flattened.index = valid_indices # Mantener el índice original

                    # Unir con el DataFrame original por el índice (id del pago)
                    df_payments_flattened = df_payments_flattened.merge(
                        df_col_flattened.add_prefix(f'{col}_'),
                        left_index=True,
                        right_index=True,
                        how='left',
                        suffixes=('', f'_{col}_dup')
                    )

                # Eliminar la columna original después de aplanarla
                df_payments_flattened = df_payments_flattened.drop(columns=[col])
                print(f"Columna '{col}' aplanada y eliminada. Nuevas columnas añadidas.")

            except Exception as e:
                print(f"ADVERTENCIA: No se pudo aplanar la columna '{col}' completamente. Error: {e}")
                print(f"Revisar la estructura JSON de '{col}' para un aplanamiento específico o manejar errores.")
        else:
            print(f"La columna '{col}' no contiene datos JSON válidos o no es un diccionario/lista en las filas seleccionadas.")
    else:
        print(f"La columna '{col}' no existe en el DataFrame original. Saltando.")


print("\nAplanamiento de campos JSON completado.")
# La línea corregida para el SyntaxError:
print(f"Nuevo tamaño del DataFrame \"df_payments_flattened\": {df_payments_flattened.shape[0]} filas, {df_payments_flattened.shape[1]} columnas.")

# --- Paso 3: Mostrar las primeras filas del DataFrame aplanado para verificar ---
print("\nPrimeras 5 filas del DataFrame 'df_payments_flattened':")
print(df_payments_flattened.head())

# --- Paso 4: Mostrar las nuevas columnas creadas ---
print("\nNuevas columnas añadidas (prefijadas por el nombre de la columna JSON original):")
# Obtener las columnas antes de aplanar (excluyendo las originales JSON que se eliminaron)
initial_non_json_cols = [c for c in df_payments.columns if c not in json_columns]
new_cols = [col for col in df_payments_flattened.columns if col not in initial_non_json_cols]
print(new_cols)

Conectando a la base de datos y cargando la tabla 'payments'...
Tabla 'payments' cargada exitosamente. Filas: 614, Columnas: 32

Iniciando aplanamiento de columnas JSON...
Aplanando columna: 'reference_document'
Columna 'reference_document' aplanada y eliminada. Nuevas columnas añadidas.
Aplanando columna: 'taxes'
Columna 'taxes' aplanada y eliminada. Nuevas columnas añadidas.
Aplanando columna: 'retentions'
Columna 'retentions' aplanada y eliminada. Nuevas columnas añadidas.
Aplanando columna: 'retentions_details'
Columna 'retentions_details' aplanada y eliminada. Nuevas columnas añadidas.
Aplanando columna: 'payments'
Columna 'payments' aplanada y eliminada. Nuevas columnas añadidas.
Aplanando columna: 'DIAN_status'
Columna 'DIAN_status' aplanada y eliminada. Nuevas columnas añadidas.
Aplanando columna: 'contact_data'
Columna 'contact_data' aplanada y eliminada. Nuevas columnas añadidas.
Aplanando columna: 'details'
Columna 'details' aplanada y eliminada. Nuevas columnas añadidas.
Ap

In [ ]:
# 1. Reconectar (o asegurar que la conexión esté activa)
try:
    conexion = get_connection()
    cursor = conexion.cursor()

    # 2. Ejecutar la consulta para describir la tabla
    cursor.execute("DESCRIBE payments;")
    columns_info = cursor.fetchall()

    print("Columnas de la tabla 'payments':")
    for column in columns_info:
        print(f"  - {column[0]} ({column[1]})")

except Exception as e:
    print(f"Ocurrió un error: {e}")
finally:
    # 3. Cerrar el cursor y la conexión de forma segura
    if 'cursor' in locals() and cursor is not None:
        cursor.close()
    if 'conexion' in locals() and conexion.is_connected():
        conexion.close()
    print("Conexión y cursor cerrados.")

✅ Conexión exitosa a la base de datos
Columnas de la tabla 'payments':
  - id (bigint unsigned)
  - document_type_id (int unsigned)
  - billing_numbering_id (int unsigned)
  - payment_type (smallint unsigned)
  - contact_id (int unsigned)
  - number (int unsigned)
  - document_date (date)
  - document_hour (time)
  - associate_invoice (tinyint(1))
  - reference_document (json)
  - advance (tinyint(1))
  - subtotal (double(16,2))
  - discount (double(16,2))
  - gross (double(16,2))
  - pending (double(16,2))
  - total (double(16,2))
  - taxes (json)
  - retentions_value (double(16,2))
  - retentions (json)
  - retentions_details (json)
  - payments (json)
  - notes (varchar(255))
  - comments (varchar(255))
  - document_status_id (smallint unsigned)
  - DIAN_status (json)
  - contact_data (json)
  - details (json)
  - extra_data (json)
  - user_id (int unsigned)
  - deleted_at (timestamp)
  - created_at (timestamp)
  - updated_at (timestamp)
Conexión y cursor cerrados.


---

---

****GLOBAL****

---

In [9]:
import pandas as pd
from sqlalchemy import create_engine

# Asegúrate de que get_engine() está definida en db_connection.py
from db_connection import get_engine

# Lista de todas las tablas que hemos analizado
tables_to_check = [
    'accounting_account_balances',
    'accounting_accounts',
    'accounting_movements',
    'accounting_voucher_items',
    'accounting_voucher_types',
    'accounting_vouchers',
    'retention_concepts',
    'retentions',
    'retentions_applied',
    'retentions_certificates',
    'taxes',
    'payments'
]

engine = None # Inicializar engine fuera del try para que sea accesible en finally

try:
    print("Conectando a la base de datos para revisar datos faltantes...")
    engine = get_engine()
    print("Conexión a la base de datos establecida.")

    for table_name in tables_to_check:
        print(f"\n--- Analizando datos faltantes en la tabla: '{table_name}' ---")
        try:
            df = pd.read_sql(f'SELECT * FROM {table_name}', engine)

            if df.empty:
                print(f"La tabla '{table_name}' está vacía. No hay datos que analizar.")
                continue

            # Calcular la cantidad de valores nulos por columna
            missing_values_count = df.isnull().sum()

            # Calcular el porcentaje de valores nulos por columna
            missing_values_percentage = (df.isnull().sum() / len(df)) * 100

            # Crear un DataFrame con el recuento y el porcentaje de nulos
            missing_data_df = pd.DataFrame({
                'Missing Count': missing_values_count,
                'Missing Percentage': missing_values_percentage
            })

            # Filtrar solo las columnas que tienen valores faltantes
            missing_data_df = missing_data_df[missing_data_df['Missing Count'] > 0].sort_values(by='Missing Percentage', ascending=False)

            if missing_data_df.empty:
                print(f"No se encontraron valores faltantes en la tabla '{table_name}'.")
            else:
                print("Valores faltantes por columna:")
                print(missing_data_df)

            # Opcional: Visualización simple para tablas más pequeñas
            # import matplotlib.pyplot as plt
            # import seaborn as sns
            # if not missing_data_df.empty:
            #     plt.figure(figsize=(10, 6))
            #     sns.barplot(x=missing_data_df.index, y='Missing Percentage', data=missing_data_df)
            #     plt.title(f'Porcentaje de Valores Faltantes en {table_name}')
            #     plt.ylabel('Porcentaje Faltante (%)')
            #     plt.xticks(rotation=45, ha='right')
            #     plt.tight_layout()
            #     plt.show()

        except Exception as e:
            print(f"Error al cargar o analizar la tabla '{table_name}': {e}")
            print(f"Asegúrate de que la tabla '{table_name}' existe y es accesible.")

except Exception as e:
    print(f"Error general de conexión a la base de datos: {e}")
finally:
    if engine:
        # Asegúrate de cerrar la conexión si es un motor que lo requiere explícitamente
        # Para SQLAlchemy con un pool, esto no siempre es necesario o deseable.
        # Pero es buena práctica si la conexión directa lo requiere.
        # En general, el `engine` de SQLAlchemy maneja el pool de conexiones.
        pass # La gestión de conexiones con `engine` de SQLAlchemy es automática.
    print("\nProceso de revisión de datos faltantes completado.")

ModuleNotFoundError: No module named 'db_connection'